# Integrating risk management tools for regional forest planning: an interactive multiobjective value at risk approach

## We demonstrate a method of incorporating multi-criteria decision making with stochastic programming

### First import required packages

In [1]:
import pandas as pd
import random
from pyomo.environ import *
import itertools
from pyomo.opt import SolverFactory
import numpy as np


### Import the data

In [2]:
forest_data = pd.read_csv("INCOME_BD_DATA.dat",index_col=False)

In [3]:
#Shuffles simulations, so that we have different inventory errors for different simulations. 
#As this is a random process, each iteration will produce slightly different results
#An error occurs, as we are vopying over data
b = list(range(0,25))
for i in forest_data.ID.unique():
    random.shuffle(b)
    c = len(forest_data.SIMULATION[forest_data["ID"]==i])
    c = c/25 #number of branches
    forest_data.SIMULATION[forest_data["ID"]== i] = b*int(c)

#Generate a list of stands based on the forest data
stands = set(forest_data["ID"])

#Generate a list of options for each stand, each stand has different management schedules possible.
options = {}
for stand in stands:
    options[stand] =list(set(forest_data[(forest_data["ID"] == stand)]["BRANCH"]))
options_sims = set(forest_data["SIMULATION"])

#Lists of the key elements we are going to examine.
income_periods = ["First Period (EUR)","Second Period (EUR)","Third Period (EUR)","Fourth Period (EUR)",
                  "Fifth Period (EUR)","Sixth Period (EUR)"]
biodiversity_periods = ["First Period (BIO)","Second Period (BIO)","Third Period (BIO)","Fourth Period (BIO)",
                       "Fifth Period (BIO)","Sixth Period (BIO)"]

# Create the optimization model

## Initialize model and create variables

In [4]:
model = ConcreteModel()

#Alternatives to be chosen
model.alternative= Var(set(forest_data.index.values),bounds=(0,1),domain=NonNegativeReals)

#Which simulations are to be relaxed
model.relaxationIncome = Var(options_sims,domain=Binary)
model.relaxationBiodiversity = Var(options_sims,domain=Binary)
relaxationNO = 100000.0

#Min income and biodiversity (over periods) for all simulations
model.minIncome = Var(options_sims)
model.minBiodiversity = Var(options_sims)

#Relaxed min income and biodiversity 
model.minIncomeRelaxed = Var()
model.minBiodiversityRelaxed = Var()

#A help variable for getting the min income in all simulations that are not relaxed
model.minIncomeRelaxed_all = Var(options_sims)
model.minBiodiversityRelaxed_all = Var(options_sims)

#Risk i.e., the proportion of simulations relaxed from calculation, separately for income and biodiversity
model.risk = Var(range(2),bounds=(0,0.999),domain=NonNegativeReals) #First risk for income, second for biodiversity

#Finally the value of the ASF scalarization when we maximize the min income and biodiversity, relaxed income and biodiversity 
#and (1-risk) for both objectives
model.asf = Var()
model.epsilon_sum = Var()
model.minINCOME1 =  Var(options_sims)
model.minINCOME2 =  Var(options_sims)
model.minINCOME3 =  Var(options_sims)
model.minINCOME4 =  Var(options_sims)
model.minINCOME5 =  Var(options_sims)
model.minINCOME6 =  Var(options_sims)

model.minBIO1 =  Var(options_sims)
model.minBIO2 =  Var(options_sims)
model.minBIO3 =  Var(options_sims)
model.minBIO4 =  Var(options_sims)
model.minBIO5 =  Var(options_sims)
model.minBIO6 =  Var(options_sims)

## Area constraint, each stand must be assigned to schedules summing to one.
Each stand must have some form of management. <p>

For each scenario, the management must be the 

In [5]:
#For each simulation and stand, a single branch needs to be selected
def onlyOneAlternativerule(model,stand):
    return sum(model.alternative[idx] 
               for idx in forest_data[(forest_data["ID"] == stand)&(forest_data["SIMULATION"] == 0)].index.values
              )==1
model.alternative_const = Constraint(stands,rule = onlyOneAlternativerule)

#Requires that each scneario has the same branch
for stand in stands:
    def onlyOneAlternativerule2(model,sim,branch):
        return sum(model.alternative[idx] -model.alternative[idd]
               for idx in forest_data[(forest_data["ID"] == stand)&(forest_data["SIMULATION"] == 0)&(forest_data["BRANCH"] == branch)].index.values
                for idd in forest_data[(forest_data["ID"] == stand)&(forest_data["SIMULATION"] == sim)&(forest_data["BRANCH"] == branch)].index.values
              )==0
    setattr(model,"onlyOneAlternativerule2"+str(stand),Constraint(options_sims,options[stand],rule=onlyOneAlternativerule2))

## Equations to evaluate the income and biodiversity for each period

In [6]:
#this section calculates the values for each of the six periods of either income or biodiversity
def incomeDummyConstraint1a(model,option_sim):
    return model.minINCOME1[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,income_periods[0]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.incomeDummy_const1a = Constraint(options_sims,rule = incomeDummyConstraint1a)

def incomeDummyConstraint2(model,option_sim):
    return model.minINCOME2[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,income_periods[1]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.incomeDummy_const2 = Constraint(options_sims,rule = incomeDummyConstraint2)
def incomeDummyConstraint3(model,option_sim):
    return model.minINCOME3[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,income_periods[2]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.incomeDummy_const3 = Constraint(options_sims,rule = incomeDummyConstraint3)
def incomeDummyConstraint4(model,option_sim):
    return model.minINCOME4[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,income_periods[3]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.incomeDummy_const4 = Constraint(options_sims,rule = incomeDummyConstraint4)
def incomeDummyConstraint5(model,option_sim):
    return model.minINCOME5[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,income_periods[4]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.incomeDummy_const5 = Constraint(options_sims,rule = incomeDummyConstraint5)
def incomeDummyConstraint6(model,option_sim):
    return model.minINCOME6[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,income_periods[5]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.incomeDummy_const6 = Constraint(options_sims,rule = incomeDummyConstraint6)

def BIODummyConstraint1(model,option_sim):
    return model.minBIO1[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,biodiversity_periods[0]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.BIODummy_const1 = Constraint(options_sims,rule = BIODummyConstraint1)

def BIODummyConstraint2(model,option_sim):
    return model.minBIO2[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,biodiversity_periods[1]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.BIODummy_const2 = Constraint(options_sims,rule = BIODummyConstraint2)
def BIODummyConstraint3(model,option_sim):
    return model.minBIO3[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,biodiversity_periods[2]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.BIODummy_const3 = Constraint(options_sims,rule = BIODummyConstraint3)
def BIODummyConstraint4(model,option_sim):
    return model.minBIO4[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,biodiversity_periods[3]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.BIODummy_const4 = Constraint(options_sims,rule = BIODummyConstraint4)
def BIODummyConstraint5(model,option_sim):
    return model.minBIO5[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,biodiversity_periods[4]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.BIODummy_const5 = Constraint(options_sims,rule = BIODummyConstraint5)
def BIODummyConstraint6(model,option_sim):
    return model.minBIO6[option_sim] == sum([model.alternative[idx]*forest_data.at[idx,biodiversity_periods[5]] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.BIODummy_const6 = Constraint(options_sims,rule = BIODummyConstraint6)


## Constraints requiring that the risk level is met, for both income and biodiversity.

In [7]:
#Risk is the proportion of either incomes or biodiversities that are relaxed 
def relaxationLimitIncome(model):
    return model.risk[0]*len(options_sims)>=summation(model.relaxationIncome)
model.incomeRelaxationLimit = Constraint(rule = relaxationLimitIncome)
def relaxationLimitBiodiversity(model):
    return model.risk[1]*len(options_sims)>=summation(model.relaxationBiodiversity)
model.biodiversityRelaxationLimit = Constraint(rule = relaxationLimitBiodiversity)


## Evaluating and constraining the income and biodiversity risk values.

In [8]:
#Min income and biodiversity must be less than incomes and biodiversities in all periods
def incomeDummyConstraint(model,option_sim,incomeperiod):
    return model.minIncome[option_sim] <= sum([model.alternative[idx]*forest_data.at[idx,incomeperiod] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.incomeDummy_const = Constraint(options_sims,income_periods,rule = incomeDummyConstraint)
def biodiversityDummyConstraint(model,option_sim,biodiversityperiod):
    return model.minBiodiversity[option_sim] <= sum([model.alternative[idx]*forest_data.at[idx,biodiversityperiod] 
                                   for idx in forest_data[forest_data["SIMULATION"]==option_sim].index.values])
model.biodiversityDummy_const = Constraint(options_sims,biodiversity_periods,rule = biodiversityDummyConstraint)

#Relaxed income and biodiversity are calculated by adding relaxationNO in relaxed simulations
def incomeDummyConstraintRelaxed_all(model,option_sim):
    return model.minIncomeRelaxed_all[option_sim] == model.minIncome[option_sim]+model.relaxationIncome[option_sim]*relaxationNO                                   
model.incomeDummyRelaxed_const_all = Constraint(options_sims,rule = incomeDummyConstraintRelaxed_all)
def biodiversityDummyConstraintRelaxed_all(model,option_sim):
    return model.minBiodiversityRelaxed_all[option_sim] == model.minBiodiversity[option_sim]+model.relaxationBiodiversity[option_sim]*relaxationNO
model.biodiversityDummyRelaxed_const_all = Constraint(options_sims,rule = biodiversityDummyConstraintRelaxed_all)
                          
#Then finally, relaxed simulations in all is taken as a minimum.    
def incomeDummyConstraintRelaxed(model,option_sim):
    return model.minIncomeRelaxed <= model.minIncomeRelaxed_all[option_sim]
model.incomeDummyRelaxed_const = Constraint(options_sims,rule = incomeDummyConstraintRelaxed)
def biodiversityDummyConstraintRelaxed(model,option_sim):
    return model.minBiodiversityRelaxed <= model.minBiodiversityRelaxed_all[option_sim]
model.biodiversityDummyRelaxed_const = Constraint(options_sims,rule = biodiversityDummyConstraintRelaxed)

## Define the Acheivement Scalarization Function problem

In [9]:
#parameter values for the nadir, ideal and reference
#Initial values are unknown, and need to be calculated
model.nadir = Param(range(6),default = 0,mutable=True)
model.ideal = Param(range(6),default=1,mutable=True)
model.reference = Param(range(6),default = 0,mutable=True)
#Add constraints which will linearize the max-min objective
#First the mean income and biodiversity
def asfDummyIncomeMean(model):
    return model.asf <= (sum(model.minIncome[option] for option in options_sims)/len(options_sims)
                         -model.reference[0])/(model.ideal[0]-model.nadir[0])
model.asfDummyIncomeMean_constraint = Constraint(rule=asfDummyIncomeMean)

def asfDummyBiodiversityMean(model):
    return model.asf <= (sum(model.minBiodiversity[option] for option in options_sims)/len(options_sims)
                         -model.reference[1])/(model.ideal[1]-model.nadir[1])
model.asfDummyBiodiversityMean_constraint = Constraint(rule=asfDummyBiodiversityMean)

#Then relaxed income and biodiversity
def asfDummyIncomeVaR(model):
    return model.asf <= (model.minIncomeRelaxed-model.reference[2])/(model.ideal[2]-model.nadir[2])
model.asfDummyIncomeMeanRelaxed_constraint = Constraint(rule=asfDummyIncomeVaR)

def asfDummyBiodiversityVaR(model):
    return model.asf <= (model.minBiodiversityRelaxed-model.reference[3])/(model.ideal[3]-model.nadir[3])
model.asfDummyBiodiversityMeanRelaxed_constraint = Constraint(rule=asfDummyBiodiversityVaR)

#Finally, (1-risk)
def asfDummyRisk(model,risk_no):
    return model.asf <= ((1-model.risk[risk_no])-
                         model.reference[4+risk_no])/(model.ideal[4+risk_no]-model.nadir[4+risk_no])
model.asfDummyRisk_constraint = Constraint(range(2),rule=asfDummyRisk)

#a function to ensure efficiency
def epsilon_sum_function(model):
    return model.epsilon_sum == (sum(model.minIncome[option] for option in options_sims)/len(options_sims)
                         -model.reference[0])/(model.ideal[0]-model.nadir[0])+(sum(model.minBiodiversity[option] for option in options_sims)/len(options_sims)
                         -model.reference[1])/(model.ideal[1]-model.nadir[1])+ (model.minIncomeRelaxed-model.reference[2])/(model.ideal[2]-model.nadir[2])+ (model.minBiodiversityRelaxed-model.reference[3])/(model.ideal[3]-model.nadir[3])
model.epsilon_sum_constraint = Constraint(rule=epsilon_sum_function)

model.objective = Objective(expr=model.asf*1000+model.epsilon_sum/1000,sense=maximize)

## A function to automate the solving process

In [10]:
def solveASF(nadir,ideal,reference,model,solver):
    for i in range(6):
        model.nadir[i] = nadir[i]
        model.ideal[i] = ideal[i]
        model.reference[i] = reference[i]
    model.preprocess() #This is a must after changing parameter value
    opt = SolverFactory(solver)
    if solver == "cbc":
        opt.options["ratio"] = 0.005
        #opt.options["threads"] = 8
    opt.solve(model,tee=False)

## Calculate Ideal and Nadir

In [11]:
relaxationNO = 1000000.0

def calculateIdealandNadir(model,solver):
    trade_off_table = []
    a= []
    b1=[]
    b2=[]
    for i in range(6):
        reference = [-relaxationNO]*6
        reference[i] = 0
        print(reference)
        solveASF([0,0,0,0,0,0],[1,1,1,1,1,1],reference,model,solver)
        trade_off_table.append([sum([value(model.minIncome[option]) for option in options_sims])/len(options_sims),
                               sum([value(model.minBiodiversity[option]) for option in options_sims])/len(options_sims),
                               value(model.minIncomeRelaxed),
                               value(model.minBiodiversityRelaxed),
                               (1-value(model.risk[0])),
                               (1-value(model.risk[1]))])
        a.append(trade_off_table[i][i])
        b1.append(sum([value(model.minIncome[option]) for option in options_sims])/len(options_sims))
        b2.append(sum([value(model.minBiodiversity[option]) for option in options_sims])/len(options_sims))
    
    print("Trade-off table:")
    print("Income, BD, VaR Income, VaR BD,1-delta for Income, 1-delta for BD")
    for objectives in trade_off_table:
        for i in objectives[:-1]:
            print(round(i,2), end=", ")
        print(round(i,2))

    return a, b1,b2, trade_off_table
        

In [12]:
a, b1,b2,trade_off_table = calculateIdealandNadir(model,"cplex")

[0, -1000000.0, -1000000.0, -1000000.0, -1000000.0, -1000000.0]
[-1000000.0, 0, -1000000.0, -1000000.0, -1000000.0, -1000000.0]
[-1000000.0, -1000000.0, 0, -1000000.0, -1000000.0, -1000000.0]
[-1000000.0, -1000000.0, -1000000.0, 0, -1000000.0, -1000000.0]
[-1000000.0, -1000000.0, -1000000.0, -1000000.0, 0, -1000000.0]
[-1000000.0, -1000000.0, -1000000.0, -1000000.0, -1000000.0, 0]
Trade-off table:
Income, BD, VaR Income, VaR BD,1-delta for Income, 1-delta for BD
5692.41, 30083.48, 5245.48, 29981.2, 0.08, 0.08
1143.56, 42910.98, 1259.1, 42690.92, 0.04, 0.04
5414.69, 32911.23, 6039.32, 33368.14, 0.04, 0.04
280.37, 42784.02, 388.02, 43229.56, 0.04, 0.04
4228.57, 41215.69, 3652.59, 41631.64, 1.0, 1.0
4107.73, 41352.13, 4782.18, 41186.32, 0.04, 0.04


In [13]:
#To aid in computability, the income and biodiveristy values are adjusted to be of similar magnitude in the optimization
#We have found rounding errors can impact the solution, and ensuring similar magintude of values avoids these types of errors
areas = sum(forest_data[(forest_data["BRANCH"] == 1)&(forest_data["SIMULATION"] == 0)].AREA.values)
multipliers = [1000/areas,0.001/areas,1000/areas,0.001/areas,1,1]

#creates the nadir and ideal vectors
b= [b1[1],b2[0],b1[1],b2[0],0.04,0.04]
b[4], b[5] =0.04, 0.04
a[4], a[5] =1, 1

## A simple function to print the results of a single optimization

In [14]:
def print_results():
    result = [round(sum([value(model.minIncome[option]) for option in options_sims])/len(options_sims),2),
                               round(sum([value(model.minBiodiversity[option]) for option in options_sims])/len(options_sims),2),
                               round(value(model.minIncomeRelaxed),2),
                               round(value(model.minBiodiversityRelaxed),2),
                               (round(1-value(model.risk[0]),2)),
                               (round(1-value(model.risk[1]),2))]
    result_ha = [result[i]*multipliers[i] for i in range(0,len(result))]
    print(result_ha)

## Optimizing - using the same reference points as in the manuscript

## Round 1- initial preferences

In [15]:
# Income, BD, Var Income, Var BD,VAR alpha INCOME, VAR alpha BD
#Reference value provided by decision maker, in per hectare values.
c = [250,.002,250,0.002,.95,.8]

#Converstion to values used by optimization, scaled to similar magnitude.
c1 = [areas*c[0]/1000,areas*c[1]*1000,areas*c[2]/1000,areas*c[3]*1000,1*c[4],1*c[5]]
solveASF(b,a,c1,model,"cplex")
print_results()

[269.5732544740992, 0.0020456944899165334, 263.55251353178517, 0.0020363907231267025, 1.0, 1.0]


## Round 2- modified preferences

In [16]:
# Income, BD, Var Income, Var BD,VAR alpha INCOME, VAR alpha BD
#Reference value provided by decision maker, in per hectare values.
c = [250,.0022,250,0.0021,.95,.95]

#Converstion to values used by optimization, scaled to similar magnitude.
c1 = [areas*c[0]/1000,areas*c[1]*1000,areas*c[2]/1000,areas*c[3]*1000,1*c[4],1*c[5]]
solveASF(b,a,c1,model,"cplex")
print_results()

[237.1607486808368, 0.0021585780676653035, 234.1907715987702, 0.0021445424279403187, 0.92, 0.92]


## Round 3 - further modified preferences

In [17]:
# Income, BD, Var Income, Var BD,VAR alpha INCOME, VAR alpha BD
#Reference value provided by decision maker, in per hectare values.
c = [350,.0,350,0.0,.95,.95]

#Converstion to values used by optimization, scaled to similar magnitude.
c1 = [areas*c[0]/1000,areas*c[1]*1000,areas*c[2]/1000,areas*c[3]*1000,1*c[4],1*c[5]]
solveASF(b,a,c1,model,"cplex")
print_results()

[302.49599255060849, 0.0015879252467185677, 298.87334333955516, 0.0015825518629856235, 0.8, 0.8]
